In [98]:
import pennylane as qml
from pennylane import numpy as np
from measurement_memory import evaluate_eigenstate_MM, GradientDescent

In [72]:
# Problem Hamiltonian
qubits = 15
H = sum([1.0*qml.PauliZ(i)@qml.PauliZ(j) for i in range(qubits) for j in range(i, qubits)])

In [74]:
p = 1 
def ansatz(params, qubits=qubits, depth=p):
    for q in range(qubits):
            qml.RY(params[q], wires=q)
    for d in range(1,depth+1):
        for q in range(qubits-1):
            qml.CNOT(wires=[q,q+1])
        for q in range(qubits):
            qml.RY(params[d*qubits+q], wires=q)

In [75]:
dev = qml.device("default.qubit", wires=qubits, shots=9999)

@qml.qnode(dev)
def sample_circuit(params):
    ansatz(params)
    return qml.counts()

In [76]:
def cost(x):
    return evaluate_eigenstate_MM(sample_circuit(x), H, memory=M, memory_states=5000)

In [77]:
init_params = np.random.rand((p+1)*qubits)
max_itr = 100
gradient_method = 'parameter_shift'

In [78]:
%%time
M = {}
params = init_params
print("itr {}, E = {}".format(0, cost(params)))
for itr in range(max_itr):
    params = GradientDescent(cost, params, gradient_method, learning_rate=0.01)
    obj_value = cost(params)
    print("itr {}, E = {}".format(itr+1, obj_value))

itr 0, E = 25.575357535753575
itr 1, E = 18.45084508450845
itr 2, E = 14.243424342434244
itr 3, E = 10.657065706570657
itr 4, E = 8.795879587958796
itr 5, E = 6.6656665666566655
itr 6, E = 5.508150815081508
itr 7, E = 4.431643164316432
itr 8, E = 3.7283728372837284
itr 9, E = 2.9834983498349836
itr 10, E = 2.3014301430143016
itr 11, E = 1.9705970597059705
itr 12, E = 1.6673667366736673
itr 13, E = 1.2527252725272526
itr 14, E = 1.0287028702870287
itr 15, E = 0.5554555455545555
itr 16, E = 0.31483148314831483
itr 17, E = 0.05300530053005301
itr 18, E = -0.16101610161016103
itr 19, E = -0.33643364336433645
itr 20, E = -0.4606460646064606
itr 21, E = -0.6418641864186418
itr 22, E = -0.8626862686268627
itr 23, E = -0.9042904290429042
itr 24, E = -1.1123112311231123
itr 25, E = -1.4657465746574658
itr 26, E = -1.3767376737673767
itr 27, E = -1.5357535753575358
itr 28, E = -1.8831883188318832
itr 29, E = -1.9887988798879888
itr 30, E = -2.0406040604060407
itr 31, E = -2.267826782678268
itr 3

In [100]:
dev = qml.device("default.qubit", wires=qubits, shots=9999)

@qml.qnode(dev, diff_method="parameter-shift")
def exp_circuit(params):
    ansatz(params)
    return qml.expval(H)

In [101]:
def cost0(x):
    return exp_circuit(x)

In [103]:
%%time
opt = qml.GradientDescentOptimizer(stepsize=0.01)

params = np.array(init_params, requires_grad=True)
print("itr {}, E = {}".format(0, cost0(params)))
for itr in range(max_itr):
    params = opt.step(cost0, params)
    obj_value = cost0(params)
    print("itr {}, E = {}".format(itr+1, obj_value))

itr 0, E = 37.15371537153716
itr 1, E = 25.803580358035802
itr 2, E = 18.521052105210522
itr 3, E = 13.826182618261827
itr 4, E = 10.553655365536553
itr 5, E = 8.288028802880289
itr 6, E = 6.787478747874788
itr 7, E = 5.432343234323432
itr 8, E = 4.3824382438243825
itr 9, E = 3.813381338133813
itr 10, E = 3.152515251525152
itr 11, E = 2.533653365336535
itr 12, E = 1.9751975197519758
itr 13, E = 1.701970197019702
itr 14, E = 1.285928592859285
itr 15, E = 0.9796979697969799
itr 16, E = 0.5508550855085504
itr 17, E = 0.43364336433643413
itr 18, E = 0.4054405440544051
itr 19, E = -0.008600860086008244
itr 20, E = -0.38243824382438296
itr 21, E = -0.5740574057405747
itr 22, E = -0.9536953695369539
itr 23, E = -0.801480148014801
itr 24, E = -1.15951595159516
itr 25, E = -1.3081308130813083
itr 26, E = -1.4313431343134313
itr 27, E = -1.5329532953295344
itr 28, E = -1.6201620162016197
itr 29, E = -1.7005700570057003
itr 30, E = -2.073607360736073
itr 31, E = -2.1752175217521748
itr 32, E = -2